In [5]:
import GetOldTweets
import pandas as pd
import numpy as np

In [22]:
tweetCriteria = GetOldTweets.got.manager.TweetCriteria().setQuerySearch('barack obama').setSince("2015-05-01").setUntil("2015-09-30").setMaxTweets(10)
tweet = GetOldTweets.got.manager.TweetManager.getTweets(tweetCriteria)[0]
printTweet(tweet)

Username: LelaMarkham
Retweets: 0
Text: @StevenDLindsey @tensharp66 @TheMightyComet @TwitchyTeam @BarackObama I personally never tried to. It's not anti-American to oppose drones.
Mentions: @StevenDLindsey @tensharp66 @TheMightyComet @TwitchyTeam @BarackObama
Hashtags: 



In [20]:
def printTweet(t):
    print("Username: %s" % t.username)
    print("Retweets: %d" % t.retweets)
    print("Text: %s" % t.text)
    print("Mentions: %s" % t.mentions)
    print("Hashtags: %s\n" % t.hashtags)